In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ckd_summary = pd.read_pickle('mimic_iv_extract/CKD_summary.pkl')
df_all_patients = pd.read_pickle("mimic_iv_extract/df_all_patients.pkl")
df_admissions = pd.read_pickle("mimic_iv_extract/df_admissions.pkl")
df_lab_events = pd.read_pickle('mimic_iv_extract/df_lab_events.pkl')
df_ckd_lab_items = pd.read_pickle('mimic_iv_extract/df_ckd_lab_items.pkl')
df_icd = pd.read_pickle("mimic_iv_extract/df_icd_codes_with_description.pkl")

# Helper Function for Age Offset Correction

In [3]:
def calculate_age_at_time(row, time_col, anchor_year_col, anchor_age_col):
    """
    Calculate the age of the patient at the time of a lab event.
    
    Parameters:
    - row (pandas.Series): A row of the DataFrame.
    - lab_time_col (str): The column name for the lab time.
    - anchor_year_col (str): The column name for the anchor year.
    - anchor_age_col (str): The column name for the anchor age.

    Returns:
    - int: The age of the patient at the time of the lab event.
    """
    # Calculate age at the time of the lab
    if pd.isnull(row[time_col]):
        return np.nan
    time_year = row[time_col].year
    anchor_year = row[anchor_year_col]
    return row[anchor_age_col] + (time_year - anchor_year)

# Helper Function for eGFR Calculation

In [4]:
def calc_eGFR(df, crt_col, sex_col, sex_flg, age_col):
    """
    Vectorized calculation approach to estimated Glomerular Filtration Rate (eGFR) using the CKD-EPI formula.
    
    Parameters:
    - df (pandas.DataFrame): Dataframe with patient data including 'gender', 'age', and 'creatinine_last'.
    
    Returns:
    - pandas.Series: A series representing the eGFR for each patient.
    """

    # Calculate numpy series for constants
    k = np.where(df[sex_col] == sex_flg, 0.7, 0.9)
    alpha = np.where(df[sex_col] == sex_flg, -0.241, -0.302)
    gender_scalar = np.where(df[sex_col] == sex_flg, 1.012, 1.0)
    
    # Calculate pandas series for Serum Creatine / k and min and max components
    scr_k_ratio = df[crt_col] / k
    min_scr_k = np.minimum(scr_k_ratio, 1)
    max_scr_k = np.maximum(scr_k_ratio, 1)
    
    # Calculate eGFR via vectorized operation
    eGFR = 142 * (min_scr_k ** alpha) * (max_scr_k ** -1.2) * (0.9938 ** age_col) * gender_scalar
    
    return eGFR

# Helper Function for CKD Stage Labeling

In [5]:
def label_CKD_stage(df, egfr_col):
    """
    Labels the stage of chronic kidney disease based on patient eGFR levels.

    Parameters:
    - egfr_srs (pandas.Series): A pandas Series containing eGFR levels of patients.

    Returns:
    - pandas.Series: A series representing the CKD stage (1-5) for each patient.
    """
    def label_patient(egfr):
        if pd.isna(egfr):
            return np.nan
        elif egfr >= 90:
            return 1
        elif egfr >= 60:
            return 2
        elif egfr >= 30:
            return 3
        elif egfr >= 15:
            return 4
        else:
            return 5

    return df[egfr_col].apply(label_patient)

In [6]:
def convert_CKD_stage(df, icd_col):
    """
    Labels the stage of chronic kidney disease based on patient eGFR levels.

    Parameters:
    - egfr_srs (pandas.Series): A pandas Series containing eGFR levels of patients.

    Returns:
    - pandas.Series: A series representing the CKD stage (1-5) for each patient.
    """
    def convert_label(icd_col):
        if icd_col == 5851:
            return 1
        elif icd_col == 5852:
            return 2
        elif icd_col == 5853:
            return 3
        elif icd_col == 5854:
            return 4
        elif icd_col == 5855:
            return 5
        elif icd_col == 5856:
            return 6
        elif icd_col == 5859:
            return -1

    return df[icd_col].apply(convert_label)

# Merge All Patient Dataframe

In [7]:
# Merge the dataframes on 'subject_id'
df_all_patient_merge = pd.merge(df_ckd_summary, df_all_patients[['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group']], on='subject_id', how='left')

# Merge Hospital Admissions Dataframe

In [8]:
# Group df_admissions by subject_id to find the earliest admittime and latest dischtime
admission_info = df_admissions.groupby('subject_id').agg(
    hadm_id_first=('hadm_id', lambda x: x.loc[df_admissions.loc[x.index, 'admittime'].idxmin()]),
    charttime_first=('admittime', 'min'),
    hadm_id_last=('hadm_id', lambda x: x.loc[df_admissions.loc[x.index, 'dischtime'].idxmax()]),
    charttime_last=('dischtime', 'max')
).reset_index()

# Merge the results into df_admissions_merge
df_admissions_merge = pd.merge(
    df_all_patient_merge,
    admission_info,
    on='subject_id',
    how='left'
)

# Rename columns
df_admissions_merge.rename(
    columns={'hadm_id_first_x': 'hadm_id_first_CKD', 'hadm_id_last_x': 'hadm_id_last_CKD',
            'admittime': 'admittime_CKD', 'dischtime': 'dischtime_CKD',
            'hadm_id_first_y': 'hadm_id_first', 'hadm_id_last_y': 'hadm_id_last',
             'charttime_first': 'admittime_first', 'charttime_last': 'dischtime_last'},
    inplace=True
)

# Calculate duration between first hospital admission and observation of change in CKD stage  
df_admissions_merge['duration'] = (df_admissions_merge['dischtime_CKD'] - df_admissions_merge['admittime_first']).dt.days

# Apply the function to calculate the discharge age and create the new column 'disch_age'
df_admissions_merge['disch_age'] = df_admissions_merge.apply(
    calculate_age_at_time, 
    time_col= 'dischtime_CKD', 
    anchor_year_col='anchor_year', 
    anchor_age_col='anchor_age', 
    axis=1
)

In [9]:
# Define remaining columns to merge from df_admissions
columns_to_merge = [
    'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 
    'insurance', 'language', 'marital_status', 'race', 'hospital_expire_flag'
]

# Perform the merge
df_admissions_merge = pd.merge(
    df_admissions_merge,
    df_admissions[['hadm_id'] + columns_to_merge],
    left_on='hadm_id_last_CKD',
    right_on='hadm_id',
    how='left'
)

# Drop the extra hadm_id column
df_admissions_merge.drop(columns=['hadm_id'], inplace=True)

# Merge Lab Events Dataframe

In [10]:
# Convert 'value' column to numeric, coercing errors to NaN
df_lab_events['value'] = pd.to_numeric(df_lab_events['value'], errors='coerce')

# Drop lab events with NaN values in the 'value' column
df_lab_events.dropna(subset=['value'], inplace=True)

# Filter df_lab_events for overlapping subject_ids
unique_subject_ids = df_admissions_merge['subject_id'].unique()
filtered_lab_events = df_lab_events[df_lab_events['subject_id'].isin(unique_subject_ids)]

# Merge DataFrames
filtered_lab_merge = filtered_lab_events.merge(df_admissions_merge, on='subject_id', how='left')

# Filter out labs after recorded after end of duration
filtered_lab_events = filtered_lab_merge[filtered_lab_merge['charttime'] <= filtered_lab_merge['dischtime_CKD']]

# Select only the original columns from filtered_df
filtered_lab_events = filtered_lab_events[df_lab_events.columns]

In [11]:
# Create a mapping dictionary from itemid to lab label
itemid_to_label = df_ckd_lab_items.set_index('itemid')['label'].to_dict()

# Initialize an empty DataFrame to store aggregated values
aggregated_lab_events = pd.DataFrame()

# Define the label for the Creatinine lab
creatinine_label = 'Creatinine'

# Loop through each unique itemid and calculate the required statistics
for itemid, group in filtered_lab_events.groupby('itemid'):
    # Sort the group by subject_id and charttime to ensure the correct order
    group_sorted = group.sort_values(by=['subject_id', 'charttime'])
    
    # Calculate the first and last values based on charttime
    first_last_df = group_sorted.groupby('subject_id').agg(
        first=('value', 'first'),
        first_time=('charttime', 'first'),
        last=('value', 'last'),
        last_time=('charttime', 'last')
    ).reset_index()

    # Sort by value for calculating other statistics
    group_sorted_by_value = group_sorted.sort_values(by=['subject_id', 'value'])

    # Calculate min, max, mean, and median values
    stats_df = group_sorted_by_value.groupby('subject_id')['value'].agg(
        min='min',
        max='max',
        mean='mean',
        median='median'
    ).reset_index()

    # Determine the lab label for the current itemid
    lab_label = itemid_to_label.get(itemid, f'itemid_{itemid}')  # Fallback to itemid if label not found

    # For Creatinine, include additional time columns
    if lab_label == creatinine_label:
        min_idx = group_sorted_by_value.groupby('subject_id')['value'].idxmin()
        max_idx = group_sorted_by_value.groupby('subject_id')['value'].idxmax()
        mean_idx = group_sorted_by_value.groupby('subject_id')['value'].apply(lambda x: (x - x.mean()).abs().idxmin())
        median_idx = group_sorted_by_value.groupby('subject_id')['value'].apply(lambda x: (x - x.median()).abs().idxmin())
        
        stats_df['min_time'] = group_sorted_by_value.loc[min_idx, 'charttime'].values
        stats_df['max_time'] = group_sorted_by_value.loc[max_idx, 'charttime'].values
        stats_df['mean_time'] = group_sorted_by_value.loc[mean_idx, 'charttime'].values
        stats_df['median_time'] = group_sorted_by_value.loc[median_idx, 'charttime'].values
        
        agg_df = pd.merge(first_last_df, stats_df, on='subject_id')
        agg_df.columns = ['subject_id'] + [f'{lab_label}_{col}' for col in agg_df.columns[1:]]
    else:
        agg_df = pd.merge(first_last_df.drop(columns=['first_time', 'last_time']), stats_df, on='subject_id')
        agg_df.columns = ['subject_id'] + [f'{lab_label}_{col}' for col in agg_df.columns[1:]]

    # Merge with the aggregated_lab_events DataFrame
    if aggregated_lab_events.empty:
        aggregated_lab_events = agg_df
    else:
        aggregated_lab_events = pd.merge(aggregated_lab_events, agg_df, on='subject_id', how='outer')

# Merge the aggregated lab events data with df_admissions_merge
df_lab_merge = pd.merge(df_admissions_merge, aggregated_lab_events, on='subject_id', how='left')

# Calculate eGFR & Update Unspecified CKD Stage Labels

In [12]:
def safe_calc_eGFR(row, crt_col, time_col):
    """
    Safely calculates eGFR for a given row of the DataFrame.

    Parameters:
    - row (pandas.Series): A row of the DataFrame.
    - crt_col (str): The name of the creatinine column.
    - lab_time_col (str): The name of the time column for the lab.

    Returns:
    - float: The calculated eGFR or NaN if data is missing.
    """
    # Check for NaN values
    if pd.isnull(row[crt_col]) or pd.isnull(row[time_col]):
        return np.nan

    # Calculate age at the time of the lab
    age_at_lab_time = calculate_age_at_time(row, time_col, 'anchor_year', 'anchor_age')
    
    # Use the previously defined calc_eGFR function
    return calc_eGFR(row, 'Creatinine_first', 'gender', 'F', age_at_lab_time)

# Perform the calculations for each eGFR value
df_lab_merge['eGFR_first'] = df_lab_merge.apply(lambda row: safe_calc_eGFR(row, 'Creatinine_first', 'Creatinine_first_time'), axis=1)
df_lab_merge['eGFR_last'] = df_lab_merge.apply(lambda row: safe_calc_eGFR(row, 'Creatinine_last', 'Creatinine_last_time'), axis=1)
df_lab_merge['eGFR_min'] = df_lab_merge.apply(lambda row: safe_calc_eGFR(row, 'Creatinine_min', 'Creatinine_min_time'), axis=1)
df_lab_merge['eGFR_max'] = df_lab_merge.apply(lambda row: safe_calc_eGFR(row, 'Creatinine_max', 'Creatinine_max_time'), axis=1)
df_lab_merge['eGFR_mean'] = df_lab_merge.apply(lambda row: safe_calc_eGFR(row, 'Creatinine_mean', 'Creatinine_mean_time'), axis=1)
df_lab_merge['eGFR_median'] = df_lab_merge.apply(lambda row: safe_calc_eGFR(row, 'Creatinine_median', 'Creatinine_median_time'), axis=1)

# Calculate the CKD stages based on eGFR
ckd_stage_first_Calc = label_CKD_stage(df_lab_merge, 'eGFR_first')
ckd_stage_last_Calc = label_CKD_stage(df_lab_merge, 'eGFR_last')

# Update CKD_stage values only if the existing value is -1 or NaN, and the calculated value is not NaN
df_lab_merge['CKD_stage_first'] = df_lab_merge.apply(
    lambda row: ckd_stage_first_Calc[row.name] if (pd.isna(row['CKD_stage_first']) or row['CKD_stage_first'] == -1) and not pd.isna(ckd_stage_first_Calc[row.name]) else row['CKD_stage_first'],
    axis=1
)

# Update CKD_stage_last only if the existing value is -1 or NaN and the new value is not NaN
df_lab_merge['CKD_stage_last'] = df_lab_merge.apply(
    lambda row: ckd_stage_last_Calc[row.name] if (pd.isna(row['CKD_stage_last']) or row['CKD_stage_last'] == -1) and not pd.isna(ckd_stage_last_Calc[row.name]) else row['CKD_stage_last'],
    axis=1
)

# Define the conditions for 'stage_delta'
conditions = [
    (df_lab_merge['CKD_stage_first'] == df_lab_merge['CKD_stage_last']),  # Condition for 0
    (df_lab_merge['CKD_stage_first'] > df_lab_merge['CKD_stage_last']),  # Condition for -1
    (df_lab_merge['CKD_stage_first'] < df_lab_merge['CKD_stage_last']),  # Condition for 1
]

# Define the corresponding values for each condition
choices = [0, -1, 1]

# Create the 'stage_delta' column
df_lab_merge['stage_delta'] = np.select(conditions, choices, default=np.nan)

# Merge ICD-9 Diagnoses Dataframe

In [13]:
# Merge ICD-9 diagnoses with associated charttimes
icd_merge = pd.merge(df_icd, df_admissions[['hadm_id', 'dischtime']], on='hadm_id', how='left')

# Merge ICD-9 diagnoses with subject's associated final CKD charttimes
icd_merge = pd.merge(icd_merge, df_lab_merge[['subject_id', 'dischtime_CKD']], on='subject_id', how='left')

# Filter out diagnoses after recorded after end of duration
filtered_icd = icd_merge[icd_merge['dischtime'] <= icd_merge['dischtime_CKD']]

# Select only the original columns
df_icd = filtered_icd[df_icd.columns]

# Extract unique long_title values from df_icd
unique_diagnoses = df_icd['long_title'].unique()

# Initialize a new DataFrame for the ICD-9 diagnosis columns with default value 0
diagnosis_df = pd.DataFrame(0, index=df_admissions_merge.index, columns=unique_diagnoses)

# Create a mapping of subject_id to diagnoses
subject_diagnoses = df_icd.groupby('subject_id')['long_title'].apply(list)

# List to store columns to be added
new_cols = []

# Populate diagnosis_df with 1 if diagnosis is present
for subject_id, diagnoses in subject_diagnoses.items():
    if subject_id in df_lab_merge['subject_id'].values:
        # Get the index of the row corresponding to the subject_id
        idx = df_lab_merge.index[df_lab_merge['subject_id'] == subject_id]
        for diagnosis in diagnoses:
            if diagnosis in diagnosis_df.columns:
                diagnosis_df.loc[idx, diagnosis] = 1
                if diagnosis not in new_cols:
                    new_cols.append(diagnosis)

# Calculate the mean for each column in diagnosis_df
means = diagnosis_df[new_cols].mean()

# Filter columns that have a mean greater than or equal to 0.005
selected_cols = means[means >= 0.005].index.tolist()

# Concatenate the necessary columns
df_icd_merge = pd.concat([df_lab_merge, diagnosis_df[selected_cols]], axis=1)

# Final Column Order for Master CKD Dataframe

In [14]:
# Define the desired column order
column_order = [
    "subject_id",
    "gender",
    "anchor_age",
    'disch_age',
    'anchor_year',
    'anchor_year_group',
    'race',
    'language',
    'marital_status',
    'insurance',
    "hadm_id_first",
    "admittime_first",
    "hadm_id_last",
    "dischtime_last",
    "hadm_id_first_CKD",
    "admittime_CKD",
    "hadm_id_last_CKD",
    "dischtime_CKD", 
    'hospital_expire_flag',
    'deathtime',
    'admission_type',
    'admit_provider_id',
    'admission_location',
    'discharge_location',
    "CKD_stage_first",
    "first_stage_icd",
    "first_long_title",
    "CKD_stage_last",
    "last_stage_icd",
    "last_long_title",
    "stage_delta",
    "duration",
    "Creatinine_first",
    "Creatinine_first_time",
    "Creatinine_last",
    "Creatinine_last_time",
    "Creatinine_min",
    "Creatinine_min_time",
    "Creatinine_max",
    "Creatinine_max_time",
    "Creatinine_mean",
    "Creatinine_mean_time",
    "Creatinine_median",
    "Creatinine_median_time",     
    "eGFR_first",
    "eGFR_last",
    "eGFR_min",
    "eGFR_max",
    "eGFR_mean",
    "eGFR_median",
]

# Get the list of all columns in the DataFrame
all_cols = df_icd_merge.columns.tolist()

# Determine the columns that were not explicitly ordered
remaining_cols = [col for col in all_cols if col not in column_order]

# Combine the desired order with the remaining columns
column_order = column_order + remaining_cols

# Reorder the DataFrame columns
CKD_master = df_icd_merge[column_order]

In [15]:
# Set pandas to display all rows and columns
pd.set_option('display.max_columns', None)

# Display the updated dataframe
display(CKD_master)

# Optionally, reset the options to their default values
pd.reset_option('display.max_columns')

subject_id gender  anchor_age  disch_age  anchor_year  \
0        10000980      F          73         78         2186   
1        10002013      F          53         59         2156   
2        10002155      F          80         83         2128   
3        10003400      F          72         75         2134   
4        10003502      F          86         86         2161   
...           ...    ...         ...        ...          ...   
14007    19997367      F          63         65         2126   
14008    19997911      F          79         86         2188   
14009    19998330      F          71         72         2177   
14010    19998497      F          82         88         2139   
14011    19999625      M          81         82         2138   

      anchor_year_group                    race language marital_status  \
0           2008 - 2010  BLACK/AFRICAN AMERICAN  ENGLISH        MARRIED   
1           2008 - 2010                   WHITE  ENGLISH         SINGLE   
2           2008 - 2010                   WHITE  ENGLISH        MARRIED   
3           2011 - 2013  BLACK/AFRICAN AMERICAN  ENGLISH        MARRIED   
4           2008 - 2010                   WHITE        ?        MARRIED   
...                 ...                     ...      ...            ...   
14007       2011 - 2013                   WHITE  ENGLISH        MARRIED   
14008       2008 - 2010  WHITE - OTHER EUROPEAN  ENGLISH        MARRIED   
14009       2011 - 2013  BLACK/AFRICAN AMERICAN  ENGLISH        MARRIED   
14010       2008 - 2010                   WHITE  ENGLISH        WIDOWED   
14011       2008 - 2010                   WHITE        ?        MARRIED   

      insurance  hadm_id_first     admittime_first  hadm_id_last  \
0      Medicare       29654838 2188-01-03 17:41:00      20897796   
1      Medicare       28603984 2156-06-28 15:53:00      28629319   
2         Other       28439444 2128-07-29 17:01:00      20345487   
3      Medicare       26090619 2134-06-06 02:25:00      23559586   
4      Medicare       22491625 2161-06-29 14:34:00      29011269   
...         ...            ...                 ...           ...   
14007  Medicare       23087270 2126-01-31 16:26:00      24169669   
14008  Medicare       23114026 2188-12-14 13:23:00      25785472   
14009     Other       27282608 2177-07-25 04:34:00      24096838   
14010  Medicare       29288061 2139-07-01 16:19:00      21557581   
14011  Medicare       27638769 2138-10-06 17:27:00      25304202   

           dischtime_last  hadm_id_first_CKD       admittime_CKD  \
0     2193-08-17 15:07:00           29654838 2188-01-03 17:41:00   
1     2167-07-05 11:44:00           24848509 2162-07-08 00:08:00   
2     2131-03-10 01:55:00           23822395 2129-08-04 12:44:00   
3     2137-09-02 17:05:00           29483621 2136-11-04 20:43:00   
4     2169-08-28 15:20:00           22491625 2161-06-29 14:34:00   
...                   ...                ...                 ...   
14007 2128-02-26 16:10:00           21508795 2127-04-02 01:03:00   
14008 2199-11-12 16:11:00           26014121 2195-11-03 19:42:00   
14009 2178-12-01 17:10:00           27282608 2177-07-25 04:34:00   
14010 2145-08-01 13:04:00           29288061 2139-07-01 16:19:00   
14011 2139-10-16 03:30:00           27638769 2138-10-06 17:27:00   

       hadm_id_last_CKD       dischtime_CKD  hospital_expire_flag  \
0              25242409 2191-04-11 16:21:00                     0   
1              24848509 2162-07-09 18:06:00                     0   
2              20345487 2131-03-10 01:55:00                     1   
3              20214994 2137-03-19 15:45:00                     0   
4              22491625 2161-07-01 16:56:00                     0   
...                 ...                 ...                   ...   
14007          24169669 2128-02-26 16:10:00                     0   
14008          26014121 2195-11-05 12:48:00                     0   
14009          24096838 2178-12-01 17:10:00                     0   
14010        

# Construct Minimal Dataframe Including only KFRE Risk Factors 

In [17]:
# Subset of columns for minimal frame including only KFRE risk factors
KFRE_cols = [
    'subject_id',
    "stage_delta",
    "duration",
    "CKD_stage_first",
    "first_stage_icd",
    "first_long_title",
    'anchor_age',
    'disch_age',
    'gender',
    "eGFR_first",
    "eGFR_last",
    "eGFR_min",
    "eGFR_max",
    "eGFR_mean",
    "eGFR_median",
    "Albumin/Creatinine,_Urine_first",
    "Albumin/Creatinine,_Urine_last",
    "Albumin/Creatinine,_Urine_min",
    "Albumin/Creatinine,_Urine_max",
    "Albumin/Creatinine,_Urine_mean",
    "Albumin/Creatinine,_Urine_median",
    "Bicarbonate_first",
    "Bicarbonate_last",
    "Bicarbonate_min",
    "Bicarbonate_max",
    "Bicarbonate_mean",
    "Bicarbonate_median",
    "Albumin_first",
    "Albumin_last",
    "Albumin_min",
    "Albumin_max",
    "Albumin_mean",
    "Albumin_median",
    "Phosphate_first",
    "Phosphate_last",
    "Phosphate_min",
    "Phosphate_max",
    "Phosphate_mean",
    "Phosphate_median",
    "Calcium,_Total_first",
    "Calcium,_Total_last",
    "Calcium,_Total_min",
    "Calcium,_Total_max",
    "Calcium,_Total_mean",
    "Calcium,_Total_median"
]

# Create a new frame with only the specified subset
CKD_KFRE = CKD_master[KFRE_cols]

# Display the resulting subset DataFrame
display(CKD_KFRE)

,subject_id,stage_delta,duration,CKD_stage_first,first_stage_icd,first_long_title,anchor_age,disch_age,gender,eGFR_first,...,Phosphate_min,Phosphate_max,Phosphate_mean,Phosphate_median,"Calcium,_Total_first","Calcium,_Total_last","Calcium,_Total_min","Calcium,_Total_max","Calcium,_Total_mean","Calcium,_Total_median"
0,10000980,1.0,1193,3.0,5853,"Chronic kidney disease, Stage III (moderate)",73,78,F,23.238428,...,3.5,5.7,4.713043,4.70,10.0,9.4,8.8,10.0,9.417021,9.40
1,10002013,0.0,2202,2.0,5859,"Chronic kidney disease, unspecified",53,59,F,76.920677,...,2.7,5.0,3.816667,3.75,8.6,8.5,8.1,9.5,8.635714,8.60
2,10002155,0.0,953,3.0,5859,"Chronic kidney disease, unspecified",80,83,F,45.760251,...,2.6,5.6,4.095349,4.10,9.4,8.4,8.0,9.6,8.843182,8.80
3,10003400,0.0,1017,3.0,5853,"Chronic kidney disease, Stage III (moderate)",72,75,F,59.856864,...,1.2,4.8,2.425758,2.15,8.5,8.1,6.7,8.9,7.970313,8.00
4,10003502,0.0,2,3.0,5859,"Chronic kidney disease, unspecified",86,86,F,40.296640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14007,19997367,0.0,755,3.0,5859,"Chronic kidney disease, unspecified",63,65,F,51.180737,...,1.6,5.6,3.395763,3.30,9.9,9.5,7.5,10.9,9.509655,9.50
14008,19997911,0.0,2516,3.0,5859,"Chronic kidney disease, unspecified",79,86,F,46.045735,...,3.2,4.5,3.740000,3.70,9.2,8.7,8.2,10.4,9.246154,9.30
14009,19998330,1.0,494,2.0,5852,"Chronic kidney disease, Stage II (mild)",71,72,F,31.862237,...,1.6,5.3,3.706250,3.85,9.5,9.4,8.8,10.2,9.440625,9.50
14010,19998497,1.0,2222,3.0,5859,"Chronic kidney disease, unspecified",82,88,F,41.055549,...,3.0,5.2,4.377778,4.35,7.7,8.3,7.6,9.1,8.422222,8.45


# Save Dataframes to Pickle Files

In [18]:
CKD_master.to_pickle("mimic_iv_extract/df_ckd_master.pkl")
CKD_KFRE.to_pickle("mimic_iv_extract/df_ckd_kfre.pkl")